In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import random

In [ ]:
# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)


In [ ]:
df = pd.read_csv("titanicc.csv")

In [ ]:
# Feature engineering
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col',
                                    'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                    'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [ ]:
columns_to_drop = ['PassengerId', 'Ticket', 'Fare', 'Name', 'Cabin']
X = df.drop(columns=columns_to_drop)
y = df['Survived']

In [ ]:
# Handle missing values
numerical_features = ['Age', 'SibSp', 'Parch', 'FamilySize']
categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title']


In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), name='l1'),
    Dropout(0.3, name='dropout1'),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), name='l2'),
    Dropout(0.3, name='dropout2'),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), name='l3'),
    Dropout(0.3, name='dropout3'),
    Dense(2, activation='softmax', name='output')
])

In [ ]:
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)


In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
23/23 [==============================] - 1s 14ms/step - loss: 0.7958 - accuracy: 0.6067 - val_loss: 0.7080 - val_accuracy: 0.7430
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6815 - accuracy: 0.7360 - val_loss: 0.6185 - val_accuracy: 0.8101
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6164 - accuracy: 0.8146 - val_loss: 0.5802 - val_accuracy: 0.8212
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6029 - accuracy: 0.8090 - val_loss: 0.5704 - val_accuracy: 0.8156
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 0.5854 - accuracy: 0.8258 - val_loss: 0.5608 - val_accuracy: 0.8212
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5888 - accuracy: 0.8062 - val_loss: 0.5479 - val_accuracy: 0.8268
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 0.5594 - accuracy: 0.8188 - val_loss: 0.5451 - val_accuracy: 0.8212
Epoch 8/100


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

6/6 [==============================] - 0s 3ms/step - loss: 0.4959 - accuracy: 0.8324
Test Accuracy: 83.24%
